In [2]:
%pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=45d31e891dad1c5cf7290833f7bf6d0fc910719e2235846e7dc2eb29ed392ad9
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [3]:
from IPython import get_ipython
from IPython.display import display
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import concurrent.futures


In [4]:
# %%
def scrape_berita(url, source):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com/",
        "Accept-Language": "id,en-US;q=0.9,en;q=0.8",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    }
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        if resp.status_code != 200:
            print(f"  ⚠️ Status code: {resp.status_code} for {url}")
            return None
        soup = BeautifulSoup(resp.content, "html.parser")

        judul = ""
        date = ""
        time_ = ""
        category = ""
        sub_category = ""
        isi = ""

        # Logika scraping berdasarkan sumber website
        if "sindonews.com" in source:
            # Untuk sindonews.com
            judul_tag = soup.select_one(".detail-title")
            judul = judul_tag.text.strip() if judul_tag else "Tidak ditemukan"

            tanggal_tag = soup.select_one(".detail-date-artikel")
            tanggal_full = tanggal_tag.text.strip() if tanggal_tag else "Tidak ditemukan"
            match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
            date = match.group(1) if match else tanggal_full
            time_ = match.group(2) if match and match.group(2) else ""

            # Konten - sindonews.com menggunakan ID
            isi_div = soup.select_one("#detail-desc")
            isi = "\n".join([p.text.strip() for p in isi_div.find_all("p") if p.text.strip()]) if isi_div else "Tidak ditemukan"

            # Kategori (Anda sebutkan Bola untuk contoh)
            # Perlu dicari selector yang lebih umum jika ingin otomatis
            category = "Bola" # Contoh tetap

            # Sub-kategori (Perlu dicari selector yang sesuai)
            sub_category = "" # Contoh tetap

        elif "tribunnews.com" in source:
            # Untuk tribunnews.com (logika scraping yang sudah ada)
            judul_tag = soup.select_one("h1")
            judul = judul_tag.text.strip() if judul_tag else "Tidak ditemukan"

            isi_paragraf = soup.select(".side-article p")
            isi = "\n".join([p.text.strip() for p in isi_paragraf if p.text.strip()]) if isi_paragraf else "Tidak ditemukan"

            # Tanggal dan waktu
            tanggal_tag = soup.select_one("time") or soup.select_one(".date")
            tanggal_full = tanggal_tag.text.strip() if tanggal_tag else "Tidak ditemukan"
            match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
            date = match.group(1) if match else tanggal_full
            time_ = match.group(2) if match and match.group(2) else ""

            # Breadcrumb: kategori dan sub-kategori
            breadcrumb = soup.select(".breadcrumb li a")
            category = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else ""
            sub_category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""

        # Tambahkan elif untuk website lain di sini
        # elif "namadomainlain.com" in source:
        #    ... logika scraping untuk namadomainlain.com ...

        else:
            # Logika default atau pesan jika sumber tidak dikenal
            print(f"  ⚠️ Sumber tidak dikenali untuk scraping: {source}")
            return None


        # Pastikan semua field terisi (minimal dengan string kosong)
        return {
            "title": judul if judul else "",
            "source": source if source else "",
            "date": date if date else "",
            "time": time_ if time_ else "",
            "category": category if category else "",
            "sub-category": sub_category if sub_category else "",
            "content": isi if isi else "",
            "url": url if url else "",
        }

    except Exception as e:
        print(f"  ❌ Gagal scraping {url}: {e}")
        return None

# Sisa kode fungsi full_rss_scrape dan contoh penggunaan tetap sama
# %%
def full_rss_scrape(list_of_rss_urls, output_csv="berita.csv", max_articles=1000):
    """
    Mengambil link artikel dari daftar URL RSS, kemudian melakukan scraping,
    dan menyimpan hasilnya ke file CSV.

    Args:
        list_of_rss_urls (list): Daftar URL RSS yang akan diambil link-nya.
        output_csv (str): Nama file CSV untuk menyimpan hasil scraping.
        max_articles (int): Jumlah maksimum artikel yang akan di-scrape dari link RSS.

    Returns:
        pandas.DataFrame: DataFrame yang berisi data artikel yang berhasil di-scrape.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.google.com/",
    }

    urls = set()
    print("➡️ Mengumpulkan link dari RSS...")
    for rss_url in list_of_rss_urls:
        try:
            print(f"  Mengambil dari: {rss_url}")
            resp = requests.get(rss_url, headers=headers, timeout=10)
            feed = feedparser.parse(resp.text)
            for entry in feed.entries:
                # Hanya tambahkan jika link tidak kosong
                if entry.link:
                  urls.add(entry.link)
            time.sleep(1) # Memberi jeda agar tidak terlalu cepat
        except Exception as e:
            print(f"  ❌ Gagal ambil dari {rss_url}: {e}")

    # Ambil sejumlah URL sesuai parameter max_articles
    urls_to_scrape = list(urls)[:max_articles]
    print(f"✅ Total link unik yang berhasil dikumpulkan: {len(urls)}")
    print(f"➡️ Akan men-scrape {len(urls_to_scrape)} artikel...")

    print("➡️ Memulai proses scraping artikel...")
    def scrape_wrapper(args):
        url, source = args
        print(f"  Scraping: {url} | Source: {source}")
        return scrape_berita(url, source)

    berita_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Menggunakan urlparse untuk mendapatkan source (nama domain)
        args_list = [(url, urlparse(url).netloc) for url in urls_to_scrape]
        results = list(executor.map(scrape_wrapper, args_list))
        for data in results:
            if data:
                berita_list.append(data)

    df = pd.DataFrame(berita_list, columns=[
        "title", "source", "date", "time", "category", "sub-category", "content", "url"
    ])

    # Simpan hasil ke CSV
    print(f"➡️ Menyimpan hasil ke {output_csv}...")
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV disimpan! Jumlah artikel berhasil di-scrape: {len(df)}")

    return df

# %%
# Contoh penggunaan fungsi full_rss_scrape:

# Ambil daftar link RSS dari file CSV (misalnya hanya untuk 'tribun')
dataset = pd.read_csv('https://raw.githubusercontent.com/gikirima/indonews-scrapper/refs/heads/main/link_scrapping.csv')
tribun_rss_links = dataset[dataset['website'] == 'tribun']['link'].tolist()
sindonews_rss_links = dataset[dataset['website'] == 'sindonews']['link'].tolist()
liputan6_rss_links = dataset[dataset['website'] == 'liputan6']['link'].tolist()
detik_rss_links = dataset[dataset['website'] == 'detik']['link'].tolist()
kapanlagi_rss_links = dataset[dataset['website'] == 'kapanlagi']['link'].tolist()
fimela_rss_links = dataset[dataset['website'] == 'fimela']['link'].tolist()
okezone_rss_links = dataset[dataset['website'] == 'okezone']['link'].tolist()
posmetro_rss_links = dataset[dataset['website'] == 'posmetro']['link'].tolist()
kompas_rss_links = dataset[dataset['website'] == 'kompas']['link'].tolist()
republika_rss_links = dataset[dataset['website'] == 'republika']['link'].tolist()
tempo_rss_links = dataset[dataset['website'] == 'tempo']['link'].tolist()


# %%
# Panggil fungsi full_rss_scrape dengan daftar link RSS
# Anda bisa menyesuaikan nama file output CSV dan jumlah maksimum artikel

# Contoh untuk sindonews
print("Scraping Sindonews...")
df_sindonews = full_rss_scrape(sindonews_rss_links, output_csv="berita_sindonews.csv", max_articles=10) # Ubah max_articles untuk pengujian

# Contoh untuk tribunnews (menggunakan logika yang sudah ada)
print("\nScraping Tribunnews...")
df_tribun = full_rss_scrape(tribun_rss_links, output_csv="berita_tribun.csv", max_articles=10) # Ubah max_articles untuk pengujian


# Jika Anda ingin melihat DataFrame hasilnya:
display(df_sindonews.head())
display(df_tribun.head())
# %%
# Anda bisa memanggil fungsi untuk website lain di sini
# df_liputan6 = full_rss_scrape(liputan6_rss_links, output_csv="berita_liputan6.csv", max_articles=500)
# ... dan seterusnya

Use code with caution

Penjelasan Perubahan:

    Parameter source di scrape_berita: Fungsi scrape_berita sekarang menerima parameter source yang berisi nama domain dari URL artikel. Parameter ini didapatkan dari fungsi full_rss_scrape menggunakan urlparse(url).netloc.
    Kondisi if-elif-else: Di awal fungsi scrape_berita, ditambahkan kondisi if "sindonews.com" in source: untuk mengecek apakah URL berasal dari sindonews.com.
    Logika Scraping Khusus Sindonews: Di dalam blok if untuk sindonews, digunakan selector CSS yang Anda berikan (.detail-title untuk judul, .detail-date-artikel untuk tanggal, dan #detail-desc untuk konten). XPath (//*[@id="detail-desc"]) tidak digunakan langsung di BeautifulSoup; sebagai gantinya, kita menggunakan selector CSS (#detail-desc) dan kemudian mencari paragraf (<p>) di dalamnya. Untuk kategori, saat ini diatur tetap "Bola" sesuai contoh Anda; Anda perlu menyesuaikannya jika ingin mengekstrak kategori secara dinamis dari halaman.
    Logika Scraping Tribunnews: Di blok elif "tribunnews.com" in source:, logika scraping yang sudah ada sebelumnya untuk tribunnews dipindahkan.
    Penanganan Sumber Lain: Anda bisa menambahkan blok elif baru untuk setiap website lain yang ingin Anda dukung, dengan menyediakan selector CSS atau metode BeautifulSoup yang sesuai untuk mengekstrak data dari struktur HTML mereka.
    Logika Default: Blok else ditambahkan untuk menangani URL dari sumber yang tidak dikenali oleh fungsi, mencetak pesan peringatan dan mengembalikan None.
    Penanganan Hasil None: Fungsi full_rss_scrape tetap menangani hasil None dari scrape_berita dengan tidak menambahkannya ke berita_list.
    Memastikan Field Terisi: Sebelum mengembalikan dictionary hasil scraping, ditambahkan pengecekan sederhana untuk memastikan setiap field memiliki nilai (setidaknya string kosong) untuk menghindari error saat membuat DataFrame.
    Perubahan di full_rss_scrape: Di dalam full_rss_scrape, saat memanggil scrape_wrapper (dan pada akhirnya scrape_berita), parameter source sekarang dikirimkan menggunakan urlparse(url).netloc.

Dengan perubahan ini, fungsi scrape_berita sekarang bisa secara otomatis menyesuaikan logika scraping berdasarkan website asal artikel. Anda hanya perlu menambahkan blok elif baru untuk setiap website lain yang ingin Anda dukung dan menentukan selector yang tepat untuk masing-masing website tersebut.

Ingatlah bahwa selector HTML bisa berubah sewaktu-waktu, jadi mungkin Anda perlu menyesuaikan kode ini jika struktur halaman website berubah.

itu aku udah ubah fungsinya menjadi parse_tribun untuk tribun news. coba kamu tambahkan parse_sindonews seperti yang telah kubuat di atas. untuk judul kan class=detail-title, date class=detail-date-artikel, dan content class=detail-desc.

coba kamu buat parse nya

Baik, mari kita modifikasi kode Anda untuk memisahkan logika parsing untuk Tribunnews dan Sindonews ke dalam fungsi terpisah (parse_tribun dan parse_sindonews), dan kemudian menggunakan kondisi if-else di fungsi utama (misalnya, kita buat fungsi parse_artikel) untuk memanggil fungsi parsing yang sesuai.

Ini akan membuat kode lebih modular dan mudah dikelola untuk setiap jenis website.

Berikut langkah-langkahnya:

    Fungsi parse_tribun: Ini adalah fungsi yang sudah Anda miliki untuk Tribunnews. Kita akan tetap menggunakannya.
    Fungsi parse_sindonews: Kita akan membuat fungsi baru khusus untuk mengurai halaman Sindonews.
    Fungsi parse_artikel (atau nama lain): Fungsi ini akan menerima URL dan nama sumber, kemudian memanggil parse_tribun atau parse_sindonews berdasarkan sumbernya.
    Modifikasi full_rss_scrape: Fungsi ini akan memanggil parse_artikel (atau fungsi baru yang dibuat di langkah 3) alih-alih memanggil scrape_berita secara langsung (atau kita bisa menamai fungsi baru tersebut scrape_berita).

Mari kita implementasikan:

from IPython import get_ipython
from IPython.display import display
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import concurrent.futures

# %%
%pip install feedparser
# %%
from IPython import get_ipython
from IPython.display import display
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import concurrent.futures

# %%
def parse_tribun(soup, url, source):
    """
    Mengurai objek BeautifulSoup untuk website tribunnews.com.

    Args:
        soup (BeautifulSoup object): Objek BeautifulSoup dari halaman yang di-scrape.
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil, None jika gagal.
    """
    try:
        judul_tag = soup.select_one("h1")
        if not judul_tag:
            print("  ⚠️ Tidak menemukan tag <h1> untuk tribunnews.com")
            return None

        isi_paragraf = soup.select(".side-article p")
        if not isi_paragraf:
            print("  ⚠️ Tidak menemukan isi artikel untuk tribunnews.com")
            return None

        judul = judul_tag.text.strip()
        isi = "\n".join([p.text.strip() for p in isi_paragraf if p.text.strip()])

        # Tanggal dan waktu
        tanggal_tag = soup.select_one("time") or soup.select_one(".date")
        tanggal_full = tanggal_tag.text.strip() if tanggal_tag else "Tidak ditemukan"
        match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
        date = match.group(1) if match else tanggal_full
        time_ = match.group(2) if match and match.group(2) else ""

        # Breadcrumb: kategori dan sub-kategori
        breadcrumb = soup.select(".breadcrumb li a")
        category = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else ""
        sub_category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""

        return {
            "title": judul,
            "source": source,
            "date": date,
            "time": time_,
            "category": category,
            "sub-category": sub_category,
            "content": isi,
            "url": url,
        }

    except Exception as e:
        print(f"  ❌ Gagal mengurai tribunnews.com {url}: {e}")
        return None

# %%
def parse_sindonews(soup, url, source):
    """
    Mengurai objek BeautifulSoup untuk website sindonews.com.

    Args:
        soup (BeautifulSoup object): Objek BeautifulSoup dari halaman yang di-scrape.
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil, None jika gagal.
    """
    try:
        judul_tag = soup.select_one(".detail-title")
        if not judul_tag:
             print("  ⚠️ Tidak menemukan .detail-title untuk sindonews.com")
             return None

        tanggal_tag = soup.select_one(".detail-date-artikel")
        if not tanggal_tag:
            print("  ⚠️ Tidak menemukan .detail-date-artikel untuk sindonews.com")
            # Masih bisa lanjut tanpa tanggal/waktu, beri nilai default
            tanggal_full = "Tidak ditemukan"
        else:
            tanggal_full = tanggal_tag.text.strip()


        isi_div = soup.select_one("#detail-desc")
        if not isi_div:
            print("  ⚠️ Tidak menemukan #detail-desc untuk sindonews.com")
            return None # Konten adalah elemen penting

        judul = judul_tag.text.strip()

        # Mengambil teks dari semua paragraf di dalam div #detail-desc
        isi = "\n".join([p.text.strip() for p in isi_div.find_all("p") if p.text.strip()])

        # Parsing tanggal dan waktu
        match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
        date = match.group(1) if match else tanggal_full
        time_ = match.group(2) if match and match.group(2) else ""

        # Kategori dan Sub-kategori (Perlu dicari selector yang sesuai di Sindonews)
        # Untuk contoh, kita bisa coba cari di breadcrumb atau meta tag
        # Jika sulit ditemukan selector umum, mungkin perlu diatur manual atau dibiarkan kosong.
        # Contoh: mencari di breadcrumb
        breadcrumb = soup.select(".breadcrumb li a") # Coba selector umum, mungkin beda
        category = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else ""
        sub_category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""

        # Jika breadcrumb di atas tidak bekerja, bisa coba meta tag atau bagian lain di HTML
        # meta_category = soup.select_one('meta[property="article:section"]')
        # category = meta_category['content'] if meta_category and 'content' in meta_category.attrs else ""

        return {
            "title": judul,
            "source": source,
            "date": date,
            "time": time_,
            "category": category, # Mungkin perlu disesuaikan
            "sub-category": sub_category, # Mungkin perlu disesuaikan
            "content": isi,
            "url": url,
        }

    except Exception as e:
        print(f"  ❌ Gagal mengurai sindonews.com {url}: {e}")
        return None

# %%
def scrape_berita(url, source):
    """
    Mengambil HTML dan memanggil fungsi parsing yang sesuai berdasarkan sumber.

    Args:
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil di-scrape dan diurai, None jika gagal.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com/",
        "Accept-Language": "id,en-US;q=0.9,en;q=0.8",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    }
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        if resp.status_code != 200:
            print(f"  ⚠️ Status code: {resp.status_code} for {url}")
            return None
        soup = BeautifulSoup(resp.content, "html.parser")

        # Pilih fungsi parsing berdasarkan sumber
        if "tribunnews.com" in source:
            return parse_tribun(soup, url, source)
        elif "sindonews.com" in source:
            return parse_sindonews(soup, url, source)
        # Tambahkan elif untuk website lain di sini
        # elif "namadomainlain.com" in source:
        #    return parse_namadomainlain(soup, url, source)
        else:
            print(f"  ⚠️ Sumber tidak dikenali untuk parsing: {source}")
            return None

    except Exception as e:
        print(f"  ❌ Gagal mengambil atau memproses HTML dari {url}: {e}")
        return None


# %%
def full_rss_scrape(list_of_rss_urls, output_csv="berita.csv", max_articles=1000):
    """
    Mengambil link artikel dari daftar URL RSS, kemudian melakukan scraping,
    dan menyimpan hasilnya ke file CSV.

    Args:
        list_of_rss_urls (list): Daftar URL RSS yang akan diambil link-nya.
        output_csv (str): Nama file CSV untuk menyimpan hasil scraping.
        max_articles (int): Jumlah maksimum artikel yang akan di-scrape dari link RSS.

    Returns:
        pandas.DataFrame: DataFrame yang berisi data artikel yang berhasil di-scrape.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.google.com/",
    }

    urls = set()
    print("➡️ Mengumpulkan link dari RSS...")
    for rss_url in list_of_rss_urls:
        try:
            print(f"  Mengambil dari: {rss_url}")
            resp = requests.get(rss_url, headers=headers, timeout=10)
            feed = feedparser.parse(resp.text)
            for entry in feed.entries:
                # Hanya tambahkan jika link tidak kosong
                if entry.link:
                  urls.add(entry.link)
            time.sleep(1) # Memberi jeda agar tidak terlalu cepat
        except Exception as e:
            print(f"  ❌ Gagal ambil dari {rss_url}: {e}")

    # Ambil sejumlah URL sesuai parameter max_articles
    urls_to_scrape = list(urls)[:max_articles]
    print(f"✅ Total link unik yang berhasil dikumpulkan: {len(urls)}")
    print(f"➡️ Akan men-scrape {len(urls_to_scrape)} artikel...")

    print("➡️ Memulai proses scraping artikel...")
    def scrape_wrapper(args):
        url, source = args
        print(f"  Scraping: {url} | Source: {source}")
        # Memanggil fungsi scrape_berita yang sudah diperbarui
        return scrape_berita(url, source)

    berita_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Menggunakan urlparse untuk mendapatkan source (nama domain)
        args_list = [(url, urlparse(url).netloc) for url in urls_to_scrape]
        results = list(executor.map(scrape_wrapper, args_list))
        for data in results:
            if data:
                berita_list.append(data)

    df = pd.DataFrame(berita_list, columns=[
        "title", "source", "date", "time", "category", "sub-category", "content", "url"
    ])

    # Simpan hasil ke CSV
    print(f"➡️ Menyimpan hasil ke {output_csv}...")
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV disimpan! Jumlah artikel berhasil di-scrape: {len(df)}")

    return df

In [16]:
# Contoh penggunaan fungsi full_rss_scrape:

# Ambil daftar link RSS dari file CSV (misalnya hanya untuk 'tribun')
dataset = pd.read_csv('https://raw.githubusercontent.com/gikirima/indonews-scrapper/refs/heads/main/link_scrapping.csv')
tribun_rss_links = dataset[dataset['website'] == 'tribun']['link'].tolist()
sindonews_rss_links = dataset[dataset['website'] == 'sindonews']['link'].tolist()
liputan6_rss_links = dataset[dataset['website'] == 'liputan6']['link'].tolist()
detik_rss_links = dataset[dataset['website'] == 'detik']['link'].tolist()
kapanlagi_rss_links = dataset[dataset['website'] == 'kapanlagi']['link'].tolist()
fimela_rss_links = dataset[dataset['website'] == 'fimela']['link'].tolist()
okezone_rss_links = dataset[dataset['website'] == 'okezone']['link'].tolist()
posmetro_rss_links = dataset[dataset['website'] == 'posmetro']['link'].tolist()
kompas_rss_links = dataset[dataset['website'] == 'kompas']['link'].tolist()
republika_rss_links = dataset[dataset['website'] == 'republika']['link'].tolist()
tempo_rss_links = dataset[dataset['website'] == 'tempo']['link'].tolist()




In [20]:
# Panggil fungsi full_rss_scrape dengan daftar link RSS
# Anda bisa menyesuaikan nama file output CSV dan jumlah maksimum artikel
df_tribun = full_rss_scrape(tribun_rss_links, output_csv="berita_tribun.csv", max_articles=500)

# Jika Anda ingin melihat DataFrame hasilnya:
display(df_tribun.head())

['https://rss.tempo.co/',
 'https://rss.tempo.co/politik',
 'https://rss.tempo.co/hukum',
 'https://rss.tempo.co/ekonomi',
 'https://rss.tempo.co/lingkungan',
 'https://rss.tempo.co/wawancara',
 'https://rss.tempo.co/investigasi',
 'https://rss.tempo.co/cekfakta',
 'https://rss.tempo.co/tokoh',
 'https://rss.tempo.co/newsletter',
 'https://rss.tempo.co/info-tempo']

In [ ]:
df_sindonews = full_rss_scrape(sindonews_rss_links, output_csv="berita_sindonews", max_articles=500)
df_liputan6 = full_rss_scrape(liputan6_rss_links, output_csv="berita_liputan6", max_articles=500)
df_detik = full_rss_scrape(detik_rss_links, output_csv="berita_detik", max_articles=500)
df_kapanlagi = full_rss_scrape(kapanlagi_rss_links, output_csv="berita_kapanlagi", max_articles=500)
df_fimela = full_rss_scrape(fimela_rss_links, output_csv="berita_fimela", max_articles=500)
df_okezone = full_rss_scrape(okezone_rss_links, output_csv="berita_okezone", max_articles=500)
df_posmetro = full_rss_scrape(posmetro_rss_links, output_csv="berita_posmetro", max_articles=500)
df_kompas = full_rss_scrape(kompas_rss_links, output_csv="berita_kompas", max_articles=500)
df_republika = full_rss_scrape(republika_rss_links, output_csv="berita_republika", max_articles=500)
df_tempo = full_rss_scrape(tempo_rss_links, output_csv="berita_tempo", max_articles=500)